# 🎯 Complex Non-Linear Function Fitting Tutorial

## **Master Non-Linear Model Training with Interactive PyTorch Implementation**

### **Main Goal:**
Train a neural network to learn the relationship **`y = 3*sin(x) + 2*cos(10x) - 0.5`** from noisy synthetic data, while understanding every step of the machine learning pipeline through interactive visualizations.

### **Key Learning Objectives:**
- **Model Complex Functions**: Handle multi-frequency sinusoidal components
- **Neural Network Design**: Build deep networks for non-linear pattern recognition
- **Fourier Analysis**: Understand frequency domain characteristics
- **Advanced Training**: Regularization, batch normalization, and optimization
- **Overfitting Analysis**: Recognize and prevent overfitting in complex models

### **Industrial Relevance:**
Apply to **vibration analysis**, **signal processing**, **time series forecasting**, and **control system identification** in industrial applications.

### **Interactive Features:**
🎮 Real-time architecture tuning | 📊 Fourier spectrum analysis | ⚡ Advanced training monitoring | 🔬 Regularization experimentation

**Target Function**: `y = 3*sin(x) + 2*cos(10x) - 0.5`

---

## 1. Import Required Libraries

We'll use PyTorch, NumPy, Matplotlib, and Scikit-learn for this tutorial.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from IPython.display import display
import ipywidgets as widgets
from ipywidgets import interact, interactive

plt.style.use('seaborn-v0_8')
np.random.seed(42)
torch.manual_seed(42)
%matplotlib widget

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

## 2. Generate Complex Synthetic Dataset

We'll create a dataset following the relationship `y = 3*sin(x) + 2*cos(10x) - 0.5` with added Gaussian noise.

In [ ]:
def generate_complex_dataset(n_samples=200, noise_std=0.5, x_range=(-4, 4)):
    x = np.random.uniform(x_range[0], x_range[1], n_samples)
    y_true = 3 * np.sin(x) + 2 * np.cos(10 * x) - 0.5
    noise = np.random.normal(0, noise_std, n_samples)
    y_noisy = y_true + noise
    return x, y_noisy, y_true

n_train, n_test = 160, 40
noise_level = 0.5
x_train, y_train, y_train_true = generate_complex_dataset(n_train, noise_level)
x_test, y_test, y_test_true = generate_complex_dataset(n_test, noise_level)

x_train_tensor = torch.FloatTensor(x_train).unsqueeze(1).to(device)
y_train_tensor = torch.FloatTensor(y_train).unsqueeze(1).to(device)
x_test_tensor = torch.FloatTensor(x_test).unsqueeze(1).to(device)
y_test_tensor = torch.FloatTensor(y_test).unsqueeze(1).to(device)

fig, ax = plt.subplots(figsize=(10, 5))
ax.scatter(x_train, y_train, alpha=0.7, label='Noisy train', color='blue')
ax.scatter(x_test, y_test, alpha=0.7, label='Noisy test', color='green')
x_plot = np.linspace(-4, 4, 500)
y_plot = 3 * np.sin(x_plot) + 2 * np.cos(10 * x_plot) - 0.5
ax.plot(x_plot, y_plot, 'r-', label='True function', linewidth=2)
ax.set_xlabel('x')
ax.set_ylabel('y')
ax.set_title('Complex Function Dataset')
ax.legend()
ax.grid(True, alpha=0.3)
plt.show()

## 3. Define Neural Network Model Architecture

A simple linear model cannot fit this function. We'll use a multi-layer neural network.

In [ ]:
class ComplexNet(nn.Module):
    def __init__(self, hidden_size=64, num_layers=3):
        super().__init__()
        layers = [nn.Linear(1, hidden_size), nn.ReLU()]
        for _ in range(num_layers-1):
            layers += [nn.Linear(hidden_size, hidden_size), nn.ReLU()]
        layers += [nn.Linear(hidden_size, 1)]
        self.net = nn.Sequential(*layers)
    def forward(self, x):
        return self.net(x)

model = ComplexNet().to(device)
print(model)

## 4. Training Loop

Let's train the model to fit the complex function.

In [ ]:
def train_model(model, x_train, y_train, x_test, y_test, lr=0.01, epochs=1000):
    optimizer = optim.Adam(model.parameters(), lr=lr)
    criterion = nn.MSELoss()
    train_losses, test_losses = [], []
    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        pred = model(x_train)
        loss = criterion(pred, y_train)
        loss.backward()
        optimizer.step()
        model.eval()
        with torch.no_grad():
            test_pred = model(x_test)
            test_loss = criterion(test_pred, y_test)
        train_losses.append(loss.item())
        test_losses.append(test_loss.item())
        if (epoch+1) % 200 == 0:
            print(f"Epoch {epoch+1}/{epochs} | Train Loss: {loss.item():.4f} | Test Loss: {test_loss.item():.4f}")
    return train_losses, test_losses

model = ComplexNet().to(device)
train_losses, test_losses = train_model(model, x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor)

## 5. Visualize Model Predictions

Let's see how well the trained model fits the data and the true function.

In [ ]:
model.eval()
x_plot = np.linspace(-4, 4, 500)
x_plot_tensor = torch.FloatTensor(x_plot).unsqueeze(1).to(device)
with torch.no_grad():
    y_pred_plot = model(x_plot_tensor).cpu().numpy().squeeze()
plt.figure(figsize=(10, 5))
plt.scatter(x_train, y_train, alpha=0.5, label='Train data', color='blue')
plt.scatter(x_test, y_test, alpha=0.5, label='Test data', color='green')
plt.plot(x_plot, 3 * np.sin(x_plot) + 2 * np.cos(10 * x_plot) - 0.5, 'r-', label='True function', linewidth=2)
plt.plot(x_plot, y_pred_plot, 'orange', label='Model prediction', linewidth=2)
plt.xlabel('x')
plt.ylabel('y')
plt.title('Model Fit to Complex Function')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

## 6. Model Evaluation

Evaluate the model using MSE, MAE, and R² metrics.

In [ ]:
with torch.no_grad():
    train_pred = model(x_train_tensor).cpu().numpy().squeeze()
    test_pred = model(x_test_tensor).cpu().numpy().squeeze()
train_mse = mean_squared_error(y_train, train_pred)
test_mse = mean_squared_error(y_test, test_pred)
train_mae = mean_absolute_error(y_train, train_pred)
test_mae = mean_absolute_error(y_test, test_pred)
train_r2 = r2_score(y_train, train_pred)
test_r2 = r2_score(y_test, test_pred)
print(f"Train MSE: {train_mse:.4f} | Test MSE: {test_mse:.4f}")
print(f"Train MAE: {train_mae:.4f} | Test MAE: {test_mae:.4f}")
print(f"Train R²: {train_r2:.4f} | Test R²: {test_r2:.4f}")

## 6.1 Regression Plot: Predicted vs True Values

Visualize the relationship between predicted and true values with a regression plot to assess model performance.

In [ ]:
# Create regression plot showing predicted vs true values
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(7, 3))

# Plot 1: Training data regression plot
min_val = min(y_train.min(), train_pred.min())
max_val = max(y_train.max(), train_pred.max())
ax1.scatter(y_train, train_pred, alpha=0.6, color='blue', s=50, edgecolors='darkblue', linewidth=0.5)
ax1.plot([min_val, max_val], [min_val, max_val], 'r--', linewidth=2, label='Perfect Prediction')
ax1.set_xlabel('True Values')
ax1.set_ylabel('Predicted Values')
ax1.set_title(f'Training Set: Predicted vs True Values\nMSE: {train_mse:.4f} | MAE: {train_mae:.4f} | R²: {train_r2:.4f}')
ax1.grid(True, alpha=0.3)
ax1.legend()

# Add correlation coefficient for training
train_corr = np.corrcoef(y_train, train_pred)[0, 1]
ax1.text(0.05, 0.95, f'Correlation: {train_corr:.4f}', transform=ax1.transAxes, 
         bbox=dict(boxstyle='round', facecolor='lightblue', alpha=0.8),
         verticalalignment='top', fontsize=10)

# Plot 2: Test data regression plot
min_val_test = min(y_test.min(), test_pred.min())
max_val_test = max(y_test.max(), test_pred.max())
ax2.scatter(y_test, test_pred, alpha=0.6, color='green', s=50, edgecolors='darkgreen', linewidth=0.5)
ax2.plot([min_val_test, max_val_test], [min_val_test, max_val_test], 'r--', linewidth=2, label='Perfect Prediction')
ax2.set_xlabel('True Values')
ax2.set_ylabel('Predicted Values')
ax2.set_title(f'Test Set: Predicted vs True Values\nMSE: {test_mse:.4f} | MAE: {test_mae:.4f} | R²: {test_r2:.4f}')
ax2.grid(True, alpha=0.3)
ax2.legend()

# Add correlation coefficient for test
test_corr = np.corrcoef(y_test, test_pred)[0, 1]
ax2.text(0.05, 0.95, f'Correlation: {test_corr:.4f}', transform=ax2.transAxes, 
         bbox=dict(boxstyle='round', facecolor='lightgreen', alpha=0.8),
         verticalalignment='top', fontsize=10)

plt.tight_layout()
plt.show()

# Print comprehensive metrics summary
print("\n" + "="*60)
print("📊 COMPREHENSIVE MODEL EVALUATION METRICS")
print("="*60)
print(f"{'Metric':<20} {'Train':<12} {'Test':<12} {'Difference':<12}")
print("-"*60)
print(f"{'MSE':<20} {train_mse:<12.4f} {test_mse:<12.4f} {abs(test_mse-train_mse):<12.4f}")
print(f"{'MAE':<20} {train_mae:<12.4f} {test_mae:<12.4f} {abs(test_mae-train_mae):<12.4f}")
print(f"{'R² Score':<20} {train_r2:<12.4f} {test_r2:<12.4f} {abs(test_r2-train_r2):<12.4f}")
print(f"{'Correlation':<20} {train_corr:<12.4f} {test_corr:<12.4f} {abs(test_corr-train_corr):<12.4f}")
print("="*60)

# Performance assessment
if abs(test_mse - train_mse) < 0.1:
    print("✅ Good generalization: Similar performance on train and test sets")
elif test_mse > train_mse * 1.5:
    print("⚠️ Possible overfitting: Test performance significantly worse than training")
else:
    print("✅ Reasonable generalization: Test performance slightly worse than training")

if test_r2 > 0.9:
    print("🎯 Excellent model performance: R² > 0.9")
elif test_r2 > 0.7:
    print("👍 Good model performance: R² > 0.7")
elif test_r2 > 0.5:
    print("📊 Fair model performance: R² > 0.5")
else:
    print("📉 Poor model performance: R² < 0.5")

## 7. Interactive Hyperparameter Tuning (Optional)

Try different network sizes and training epochs interactively.

In [ ]:
def interactive_training(hidden_size=64, num_layers=3, lr=0.01, epochs=1000):
    model = ComplexNet(hidden_size=hidden_size, num_layers=num_layers).to(device)
    train_losses, test_losses = train_model(model, x_train_tensor, y_train_tensor, x_test_tensor, y_test_tensor, lr=lr, epochs=epochs)
    model.eval()
    x_plot = np.linspace(-4, 4, 500)
    x_plot_tensor = torch.FloatTensor(x_plot).unsqueeze(1).to(device)
    with torch.no_grad():
        y_pred_plot = model(x_plot_tensor).cpu().numpy().squeeze()
    
    # Calculate metrics for display
    with torch.no_grad():
        train_pred = model(x_train_tensor).cpu().numpy().squeeze()
        test_pred = model(x_test_tensor).cpu().numpy().squeeze()
    
    train_mse = mean_squared_error(y_train, train_pred)
    test_mse = mean_squared_error(y_test, test_pred)
    train_r2 = r2_score(y_train, train_pred)
    test_r2 = r2_score(y_test, test_pred)
    
    # Create subplots for function fit and learning curves
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot 1: Function fit with metrics
    ax1.plot(x_plot, 3 * np.sin(x_plot) + 2 * np.cos(10 * x_plot) - 0.5, 'r-', label='True function', linewidth=2)
    ax1.plot(x_plot, y_pred_plot, 'orange', label='Model prediction', linewidth=2)
    ax1.scatter(x_train, y_train, alpha=0.4, s=20, color='blue', label='Train data')
    ax1.scatter(x_test, y_test, alpha=0.4, s=20, color='green', label='Test data')
    ax1.set_xlabel('x')
    ax1.set_ylabel('y')
    ax1.set_title(f'Interactive Model Fit\nTrain R²: {train_r2:.3f}, MSE: {train_mse:.3f}\nTest R²: {test_r2:.3f}, MSE: {test_mse:.3f}')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot 2: Learning curves with final metrics
    ax2.plot(train_losses, label='Train Loss', color='blue', linewidth=2)
    ax2.plot(test_losses, label='Test Loss', color='red', linewidth=2)
    ax2.set_xlabel('Epochs')
    ax2.set_ylabel('Loss (MSE)')
    ax2.set_title(f'Learning Curves\nFinal Train MSE: {train_mse:.3f}\nFinal Test MSE: {test_mse:.3f}')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    ax2.set_yscale('log')  # Log scale for better visualization
    
    # Add text box with all metrics
    metrics_text = f'Train: R²={train_r2:.3f}, MSE={train_mse:.3f}\nTest: R²={test_r2:.3f}, MSE={test_mse:.3f}'
    ax2.text(0.02, 0.98, metrics_text, transform=ax2.transAxes, 
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.8),
             verticalalignment='top', fontsize=10)
    
    plt.tight_layout()
    plt.show()

interact(interactive_training, hidden_size=(16, 256, 16), num_layers=(2, 5), lr=(0.001, 0.05, 0.001), epochs=(500, 2000, 100));

## 🎯 Key Takeaways

- Neural networks can fit highly non-linear, multi-frequency functions.
- Model depth and width are crucial for capturing complex patterns.
- Overfitting can occur if the model is too large or trained too long.
- Interactive tuning helps find the best architecture for your data.

---

**Try modifying the function or adding more noise to further challenge your model!**